<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW5/HW5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [4]:
# 定義分割比例
train_split, valid_split = ['train[:90%]', 'train[90%:]']

# 定義 parse_fn
def parse_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.0  # 將圖像正規化到 [0, 1]
    y = tf.one_hot(dataset['label'], 10)  # MNIST 的標籤有 10 個類別
    return x, y

# 加載數據集
train_data, info = tfds.load("mnist", split=train_split, with_info=True)
valid_data = tfds.load("mnist", split=valid_split)
test_data = tfds.load("mnist", split="test")

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
train_num = info.splits['train[:90%]'].num_examples  # 訓練資料數量

# 處理訓練資料
train_data = train_data.shuffle(train_num)\
                       .map(parse_fn, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size)\
                       .prefetch(buffer_size=AUTOTUNE)

# 處理驗證資料
valid_data = valid_data.map(parse_fn, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size)\
                       .prefetch(buffer_size=AUTOTUNE)

# 處理測試資料
test_data = test_data.map(parse_fn, num_parallel_calls=AUTOTUNE)\
                     .batch(batch_size)\
                     .prefetch(buffer_size=AUTOTUNE)

In [6]:
# 確認資料流程正常
for features, labels in train_data.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

Features shape: (64, 28, 28, 1)
Labels shape: (64, 10)


In [7]:
# 定義輸入層
inputs = keras.Input(shape=(28, 28, 1), name='mnist_input')  # MNIST 是 28x28 灰階圖像

# 添加隱藏層
x = layers.Flatten()(inputs)  # 將圖像展平
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)  # 避免過擬合
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# 定義輸出層
outputs = layers.Dense(10, activation='softmax', name='mnist_output')(x)  # MNIST 有 10 類別

# 建立模型
model_mnist = keras.Model(inputs, outputs, name='mnist_model')

# 查看模型摘要
model_mnist.summary()

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mnist_input (InputLayer)             │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mnist_output (Dense)                 │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,146 (434.16 KB)

 Trainable params: 111,146 (434.16 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# 創建儲存權重目錄
model_dir = 'mnist-logs/models'
os.makedirs(model_dir, exist_ok=True)

# 建立 Callback function
log_dir = os.path.join('mnist-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(model_dir, 'Best-model-1.keras'),  # 修改副檔名為 .keras
    monitor='val_categorical_accuracy',
    save_best_only=True,
    mode='max'
)

In [9]:
# 設定優化器、損失函數和指標
model_mnist.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

# 訓練模型
history_mnist = model_mnist.fit(
    train_data,
    epochs=10,  # 為了快速測試，設置少一點 epoch，可根據需求調整
    validation_data=valid_data,
    callbacks=[model_cbk, model_mckp]
)

Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - categorical_accuracy: 0.7480 - loss: 0.7764 - val_categorical_accuracy: 0.9467 - val_loss: 0.1839
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - categorical_accuracy: 0.9453 - loss: 0.1926 - val_categorical_accuracy: 0.9655 - val_loss: 0.1265
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - categorical_accuracy: 0.9622 - loss: 0.1337 - val_categorical_accuracy: 0.9685 - val_loss: 0.1127
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - categorical_accuracy: 0.9708 - loss: 0.1040 - val_categorical_accuracy: 0.9677 - val_loss: 0.1172
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - categorical_accuracy: 0.9755 - loss: 0.0848 - val_categorical_accuracy: 0.9745 - val_loss: 0.0959
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - categorical_accuracy: 0.9791 - loss: 0.0695 - val_categorical_accuracy: 0.9737 - val_loss: 0.0936
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - categorical_accuracy: 0.9830 - lo

In [11]:
loss, acc = model_mnist.evaluate(test_data)
print('\nModel Accuracy: {}%'.format(acc))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - categorical_accuracy: 0.9757 - loss: 0.0940

Model Accuracy: 0.9735999703407288%


In [12]:
loss = [loss]
acc = [acc]

dict = {"test_Loss": loss,
        "test_Accuracy": acc}

print(pd.DataFrame(dict))

   test_Loss  test_Accuracy
0   0.097954         0.9736
